In [1]:
!nvidia-smi

Thu Apr 10 11:07:43 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install -q -U bitsandbytes
!pip install sentencepiece
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 50.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.tom

First let's load the model we are going to use - GPT-neo-x-20B! Note that the model itself is around 40GB in half precision

In [ ]:
from datasets import load_dataset

data = load_dataset("Coffeeamp/lawqa")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

law_qa_dataset.jsonl:   0%|          | 0.00/32.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14819 [00:00<?, ? examples/s]

In [4]:
data

DatasetDict({
    train: Dataset({
        features: ['question', 'precedent', 'answer'],
        num_rows: 14819
    })
})

In [5]:
data = data.map(
    lambda x: {'text': f"### 질문: {x['question']}\n\n### 답변: {x['answer']}<|endoftext|>" }
)

Map:   0%|          | 0/14819 [00:00<?, ? examples/s]

In [6]:
data['train'][0]['text']

'### 질문: 최저임금제도란 무엇이며, 최저임금액은 어떻게 결정되는지요?\n\n### 답변: 최저임금제도란 국가가 임금액의 최저한도를 정하여 사용자에게 이를 준수하도록 강제하는 제도를 말합니다. 그러므로 최저임금액이 결정·고시되면 사용자는 근로자와 합의하여 최저임금액보다 낮은 임금을 지급하기로 약정하더라도 그것은 당연히 무효가 되며, 고용노동부장관이 고시한 최저임금액 이상을 지급하여야 합니다(최저임금법 제6조). 최저임금제도의 적용대상은 근로자를 사용하는 모든 사업 또는 사업장에 적용되며, 상용근로자 뿐만 아니라 임시근로자나 일용근로자, 시간제근로자 등 모든 근로자에게 적용됩니다. 다만, 동거의 친족만을 사용하는 사업과 가사사용인에 대하여는 적용하지 아니하고, 선원법의 적용을 받는 선원 및 선원을 사용하는 선박의 소유자에 대하여는 이를 적용하지 아니합니다(같은 법 제3조). 그리고 수습사용 중에 있는 자로서 수습사용한 날부터 3월 이내인 근로자는 시간급 최저임금액의 90%를 지급할 수 있고, 사용자가 고용노동부장관의 인가를 받은 감시 또는 단속적으로 근로에 종사하는 자(수위, 경비원, 자가용운전기사 등)는 시간급 최저임금액의 80%를 지급할 수 있습니다(같은 법 제5조 제2항, 같은 법 시행령 제3조).그러나 같은 법 시행령 제6조는 사용자는 고용노동부장관의 인가를 받아 ‘근로자의 정신 또는 신체의 장애가 당해 근로자를 종사시키고자 하는 업무의 수행에 직접적으로 현저한 지장을 주는 것이 명백하다고 인정되는 자’에 대하여는 최저임금의 적용을 제외할 수 있도록 규정하고 있습니다.고용노동부장관은 매년 3월 31일까지 근로자위원, 사용자위원, 공익위원 등으로 구성된 최저임금심의위원회에 최저임금에 관한 심의를 요청하여야 하고, 동 위원회에서는 근로자의 생계비, 유사근로자의 임금, 노동생산성 및 소득분배율 등을 고려하여 최저임금안을 심의하며, 심의위원회로부터 최저임금안을 제출받은 때에는 지체없이 사업의 종류별 최저임금안 및 적용사업의 범위를 고시하여야 하며, 매년 8월

In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "EleutherAI/polyglot-ko-5.8b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

tokenizer_config.json:   0%|          | 0.00/164 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/36.8k [00:00<?, ?B/s]

...

model-00011-of-00013.safetensors:   0%|          | 0.00/952M [00:00<?, ?B/s]

model-00012-of-00013.safetensors:   0%|          | 0.00/948M [00:00<?, ?B/s]

model-00013-of-00013.safetensors:   0%|          | 0.00/515M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Then we have to apply some preprocessing to the model to prepare it for training. For that use the `prepare_model_for_kbit_training` method from PEFT.

In [8]:
data = data.map(lambda samples: tokenizer(samples["text"], truncation=True, max_length=2048), batched=True)

Map:   0%|          | 0/14819 [00:00<?, ? examples/s]

In [9]:
data['train'][0]['text']

'### 질문: 최저임금제도란 무엇이며, 최저임금액은 어떻게 결정되는지요?\n\n### 답변: 최저임금제도란 국가가 임금액의 최저한도를 정하여 사용자에게 이를 준수하도록 강제하는 제도를 말합니다. 그러므로 최저임금액이 결정·고시되면 사용자는 근로자와 합의하여 최저임금액보다 낮은 임금을 지급하기로 약정하더라도 그것은 당연히 무효가 되며, 고용노동부장관이 고시한 최저임금액 이상을 지급하여야 합니다(최저임금법 제6조). 최저임금제도의 적용대상은 근로자를 사용하는 모든 사업 또는 사업장에 적용되며, 상용근로자 뿐만 아니라 임시근로자나 일용근로자, 시간제근로자 등 모든 근로자에게 적용됩니다. 다만, 동거의 친족만을 사용하는 사업과 가사사용인에 대하여는 적용하지 아니하고, 선원법의 적용을 받는 선원 및 선원을 사용하는 선박의 소유자에 대하여는 이를 적용하지 아니합니다(같은 법 제3조). 그리고 수습사용 중에 있는 자로서 수습사용한 날부터 3월 이내인 근로자는 시간급 최저임금액의 90%를 지급할 수 있고, 사용자가 고용노동부장관의 인가를 받은 감시 또는 단속적으로 근로에 종사하는 자(수위, 경비원, 자가용운전기사 등)는 시간급 최저임금액의 80%를 지급할 수 있습니다(같은 법 제5조 제2항, 같은 법 시행령 제3조).그러나 같은 법 시행령 제6조는 사용자는 고용노동부장관의 인가를 받아 ‘근로자의 정신 또는 신체의 장애가 당해 근로자를 종사시키고자 하는 업무의 수행에 직접적으로 현저한 지장을 주는 것이 명백하다고 인정되는 자’에 대하여는 최저임금의 적용을 제외할 수 있도록 규정하고 있습니다.고용노동부장관은 매년 3월 31일까지 근로자위원, 사용자위원, 공익위원 등으로 구성된 최저임금심의위원회에 최저임금에 관한 심의를 요청하여야 하고, 동 위원회에서는 근로자의 생계비, 유사근로자의 임금, 노동생산성 및 소득분배율 등을 고려하여 최저임금안을 심의하며, 심의위원회로부터 최저임금안을 제출받은 때에는 지체없이 사업의 종류별 최저임금안 및 적용사업의 범위를 고시하여야 하며, 매년 8월

In [10]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [11]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [12]:
from os.path import join
from peft import LoraConfig, get_peft_model, PeftModel

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 3670016 || all params: 3070156800 || trainable%: 0.11953838970048696


Let's load a common dataset, english quotes, to fine tune our model on famous quotes.

In [13]:
!nvidia-smi

Thu Apr 10 11:16:15 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   51C    P0             28W /   70W |    5838MiB /  15360MiB |      6%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Run the cell below to run the training! For the sake of the demo, we just ran it for few steps just to showcase how to use this integration with existing tools on the HF ecosystem.

In [14]:
import transformers

# tokenizer.pad_token_id = (
#     0  # unk. we want this to be different from the eos token
# )
# tokenizer.padding_side = "left"

tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        # warmup_steps=200,
        max_steps=100, ## 초소형만 학습: 10 step = 20개 샘플만 학습.
        learning_rate=1e-4,
        fp16=True,
        logging_steps=10,
        output_dir="outputs",
        report_to="none" # api키 없이
        #optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,1.820200
20,2.000900
30,1.899200
40,1.742800
50,1.670900
60,1.684600
70,1.596600
80,1.449000
90,1.466700
100,1.618500


TrainOutput(global_step=100, training_loss=1.69492995262146, metrics={'train_runtime': 391.185, 'train_samples_per_second': 0.511, 'train_steps_per_second': 0.256, 'total_flos': 4565324347539456.0, 'train_loss': 1.69492995262146, 'epoch': 0.01349527665317139})

In [15]:
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [16]:
model.save_pretrained("./saved/doctor/5.8B/weight_5epoch")

In [17]:
def remove_suffix(input_string, suffix):
    if suffix and input_string.endswith(suffix):
        return input_string.rsplit(suffix, 1)[0]
    return input_string

In [18]:
from transformers import StoppingCriteria, StoppingCriteriaList

class StoppingCriteriaSub(StoppingCriteria):

    def __init__(self, stops = [], encounters=1):
        super().__init__()
        self.stops = [stop for stop in stops]

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        for stop in self.stops:
            if torch.all((stop == input_ids[0][-len(stop):])).item():
                return True

        return False

stop_words = ["\nHuman:"]
stop_words_ids = [tokenizer(stop_word, return_tensors='pt')['input_ids'].squeeze() for stop_word in stop_words]
stopping_criteria = StoppingCriteriaList([StoppingCriteriaSub(stops=stop_words_ids)])

In [19]:
chat_history = ''

In [40]:
def gen(x):
    global chat_history
    prefix = chat_history +('\n' if chat_history !='' else '')
    gened = model.generate(
        **tokenizer(
            f"{prefix}Human: {x}\nRemon:",
            return_tensors='pt',
            return_token_type_ids=False
        ),
        max_new_tokens=256,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
        stopping_criteria=stopping_criteria
    )
    result = tokenizer.decode(gened[0])
    chat_history = remove_suffix(result, "\nHuman:")
    print(chat_history)

In [41]:
gen("전세 사기 피해 대처법은?")

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:682: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:2334: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [46]:
print(type(model.device))  # 👇 이런 식으로 나와야 적용된 것
# <class 'peft.peft_model.PeftModelForCausalLM'>
print(model.peft_config)
# 'default': LoraConfig(...) 형태로 나와야 적용된 상태


<class 'torch.device'>
{'default': LoraConfig(task_type='CAUSAL_LM', peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='EleutherAI/polyglot-ko-5.8b', revision=None, inference_mode=False, r=8, target_modules={'query_key_value'}, exclude_modules=None, lora_alpha=32, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False)}


In [43]:
from peft import PeftModel
print(type(model))


<class 'peft.peft_model.PeftModelForCausalLM'>


In [44]:
def gen(question):
    prompt = f"질문: {question}\n답변:"
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)

    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_new_tokens=512,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

    result = tokenizer.decode(output[0], skip_special_tokens=True)
    return result.split("답변:")[-1].strip()


In [45]:
gen("전세 사기 피해 대처법은?")

'집주인의 신분증을 확인하고, 전세 계약시에는 집주인의 인감증명서가 첨부된 위임장을 받은 후, 집주인에게 연락하여 계약사항을 확인하여야 합니다. 또한, 잔금시에는 등기사항증명서를 확인하고, 전입신고와 함께 확정일자를 받아야 합니다.'

In [ ]:
	# Repository 생성 & model upload
REPO_NAME = "remon-polyglot-12.8b-qlora" # ex) 'my-bert-fine-tuned'
AUTH_TOKEN = "" # <https://huggingface.co/settings/token>

## Upload to Huggingface Hub
model.push_to_hub(
    REPO_NAME,
    use_temp_dir=True,
    use_auth_token=AUTH_TOKEN
)
tokenizer.push_to_hub(
    REPO_NAME,
    use_temp_dir=True,
    use_auth_token=AUTH_TOKEN
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:232: UserWarning: About to update multiple times the same file in the same commit: 'adapter_config.json'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:232: UserWarning: About to update multiple times the same file in the same commit: 'adapter_model.bin'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:232: UserWarning: About to update multiple times the same file in the same commit: 'README.md'. This can cause undesired inconsistencies in your repo.
  warnings.warn(


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/26.2M [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/26.2M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:232: UserWarning: About to update multiple times the same file in the same commit: 'tokenizer.json'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:232: UserWarning: About to update multiple times the same file in the same commit: 'tokenizer_config.json'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:232: UserWarning: About to update multiple times the same file in the same commit: 'special_tokens_map.json'. This can cause undesired inconsistencies in your repo.
  warnings.warn(


CommitInfo(commit_url='https://huggingface.co/myeolinmalchi/remon-polyglot-12.8b-qlora/commit/911c8a7c9aaa99beb752458a136e5a927794f1ec', commit_message='Upload tokenizer', commit_description='', oid='911c8a7c9aaa99beb752458a136e5a927794f1ec', pr_url=None, pr_revision=None, pr_num=None)